# Exercise Progress

#### Katriona Goldmann

This script looks at the exercise over time. Specifically, I am training for a half marathon so I want to compare logged runs to my plan to determine if I am on track. 

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
import datetime
import os

----

## Read in the data

In [2]:
ex = pd.read_csv("./Outputs/exercise.csv")

So lets look at the exercise completed:

In [3]:
ex.head()

,Unnamed: 0,activityId,activityParentId,activityParentName,calories,description,distance,duration,hasStartTime,isFavorite,lastModified,logId,name,startDate,startTime,steps
0,0,52000,52000,Yoga,104,NaN,NaN,1800000,True,False,2018-12-27T23:10:34.000Z,18875096545,Yoga,2018-12-27,22:40,239
1,0,90009,90009,Run,346,Running - 5 mph (12 min/mile),4.352986,2893037,True,False,2018-12-31T13:33:42.000Z,18936842588,Run,2018-12-31,12:17,7313
2,0,90009,90009,Run,111,Running - 5 mph (12 min/mile),1.304879,858000,True,False,2019-01-16T13:28:31.000Z,19144979937,Run,2019-01-15,17:50,2105
3,0,90019,90019,Treadmill,181,2mph,1.615564,900000,True,False,2019-01-18T08:14:33.000Z,19297289610,Treadmill,2019-01-18,07:43,2745
4,0,90009,90009,Run,259,Running - 5 mph (12 min/mile),3.106856,2000000,True,False,2019-01-24T17:09:35.000Z,19419790003,Run,2019-01-21,18:12,5169


You can see in January there is mainly running and yoga. This is because I unfortunately broke two fingers and was in a cast so unfortunately my exercise was pretty limited. 

-----

## Running Plan

So lets look specifically at running. 

In [4]:
runs = ex.loc[(ex.activityParentName == 'Run') | (ex.activityParentName == 'Treadmill') , ['distance', 'duration', 'calories', 'steps', 'startDate', 'startTime'] ]
runs['total_run'] = runs['distance'].cumsum()
runs.head()

,distance,duration,calories,steps,startDate,startTime,total_run
1,4.352986,2893037,346,7313,2018-12-31,12:17,4.352986
2,1.304879,858000,111,2105,2019-01-15,17:50,5.657865
3,1.615564,900000,181,2745,2019-01-18,07:43,7.273429
4,3.106856,2000000,259,5169,2019-01-21,18:12,10.380285
5,1.242742,923000,107,2020,2019-01-23,17:36,11.623027


Now lets load in the running plan. I created this using the [Nike Run Club app](https://www.nike.com/gb/en_gb/c/nike-plus/running-app-gps) then exported to a csv, which I would recommend, its super easy to create a plan using this. Although rather annoyingly, fitbit doesn't sync with this app currently. 

But however you decide to plan all you need is a csv with a distance, date and time column. 

In [5]:
plan = pd.read_csv("./Inputs/Running_Plan.csv")
plan['total_dist'] = plan['distance (km)'].cumsum()
plan.head()

,date,dow,day,week,goal,type,notes,distance (km),Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,total_dist
0,15/01/19,3,Tuesday,18,NaN,first run,NaN,2.0,NaN,NaN,NaN,NaN,2.0
1,16/01/19,4,Wednesday,18,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,2.0
2,17/01/19,5,Thursday,18,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,2.0
3,18/01/19,6,Friday,18,NaN,benchmark,15 mins,2.0,NaN,NaN,NaN,NaN,4.0
4,19/01/19,7,Saturday,18,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,4.0


In [6]:
plan['text'] = 'run so far = ' + round(plan['total_dist']).map(str) + 'km, To run today=' + plan['distance (km)'].map(str) + 'km'

Reformat some of the columns as dates and convert to the correct units. 

In [7]:
# Format the dates as time stamps
plan['date_stamp'] = [datetime.datetime.strptime(x, '%d/%m/%y') for x in plan['date']] 
runs['date_stamp']  = [datetime.datetime.strptime(x, '%Y-%m-%d') for x in runs['startDate']] 
plan['datec'] = [x.strftime('%d/%m/%Y') for x in plan['date_stamp']]
runs['datec'] = [x.strftime('%d/%m/%Y') for x in runs['date_stamp']]

# Convert to metric units
runs['total_run'] = runs['total_run']/0.62137119
runs['secs'] = runs['duration']/(1000)

plan

,date,dow,day,week,goal,type,notes,distance (km),Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,total_dist,text,date_stamp,datec
0,15/01/19,3,Tuesday,18,NaN,first run,NaN,2.0,NaN,NaN,NaN,NaN,2.0,"run so far = 2.0km, To run today=2.0km",2019-01-15,15/01/2019
1,16/01/19,4,Wednesday,18,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,2.0,"run so far = 2.0km, To run today=0.0km",2019-01-16,16/01/2019
2,17/01/19,5,Thursday,18,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,2.0,"run so far = 2.0km, To run today=0.0km",2019-01-17,17/01/2019
3,18/01/19,6,Friday,18,NaN,benchmark,15 mins,2.0,NaN,NaN,NaN,NaN,4.0,"run so far = 4.0km, To run today=2.0km",2019-01-18,18/01/2019
4,19/01/19,7,Saturday,18,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,4.0,"run so far = 4.0km, To run today=0.0km",2019-01-19,19/01/2019
5,20/01/19,1,Sunday,18,NaN,recovery,NaN,2.0,NaN,NaN,NaN,NaN,6.0,"run so far = 6.0km, To run today=2.0km",2019-01-20,20/01/2019
6,21/01/19,2,Monday,17,NaN,long run,NaN,5.0,NaN,NaN,NaN,NaN,11.0,"run so far = 11.0km, To run today=5.0km",2019-01-21,21/01/2019
7,22/01/19,3,Tuesday,17,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,11.0,"run so far = 11.0km, To run today=0.0km",2019-01-22,22/01/2019
8,23/01/19,4,Wednesday,17,NaN,benchmark,15 mins,2.0,NaN,NaN,NaN,NaN,13.0,"run so far = 13.0km, To run today=2.0km",2019-01-23,23/01/2019
9,24/01/19,5,Thursday,17,NaN,rest,NaN,0.0,NaN,NaN,NaN,NaN,13.0,"run so far = 13.0km, To run today=0.0km",2019-01-24,24/01/2019


-----

# Plot Runs

Create a combined df to determine how things are looking comparing the plan to logged runs

In [8]:
combine = pd.merge(plan, runs, how='inner', left_on='datec', right_on='datec')
combine['cumsum'] = combine['total_run'].cumsum()
combine['on_track'] = 0
combine.loc[combine['total_run'].cumsum() >= combine['total_dist'] , 'on_track'] = 1
combine['date_stamp'] = [datetime.datetime.strptime(str(x), "%d/%m/%y") for x in combine['date']]

Now let's plot!

In [14]:
temp = combine.iloc[[-1]]
temp['date_stamp'].values[-1] =  pd.Timestamp(datetime.datetime.today().date())

combine = combine.append(temp, ignore_index=False, verify_integrity=False, sort=None)

combine

,date,dow,day,week,goal,type,notes,distance (km),Unnamed: 8,Unnamed: 9,...,calories,steps,startDate,startTime,total_run,date_stamp_y,secs,cumsum,on_track,date_stamp
0,15/01/19,3,Tuesday,18,NaN,first run,NaN,2.0,NaN,NaN,...,111,2105,2019-01-15,17:50,9.105451,2019-01-15,858.0,9.105451,1,2019-01-15
1,18/01/19,6,Friday,18,NaN,benchmark,15 mins,2.0,NaN,NaN,...,181,2745,2019-01-18,07:43,11.705449,2019-01-18,900.0,20.810900,1,2019-01-18
2,21/01/19,2,Monday,17,NaN,long run,NaN,5.0,NaN,NaN,...,259,5169,2019-01-21,18:12,16.705449,2019-01-21,2000.0,37.516350,1,2019-01-21
3,23/01/19,4,Wednesday,17,NaN,benchmark,15 mins,2.0,NaN,NaN,...,107,2020,2019-01-23,17:36,18.705449,2019-01-23,923.0,56.221799,1,2019-01-23
4,23/01/19,4,Wednesday,17,NaN,benchmark,15 mins,2.0,NaN,NaN,...,73,1312,2019-01-23,18:39,20.005448,2019-01-23,633.0,76.227247,1,2019-01-23
5,02/02/19,7,Saturday,16,NaN,rest,NaN,0.0,NaN,NaN,...,134,2552,2019-02-02,10:11,22.505448,2019-02-02,931.0,98.732695,1,2019-02-02
6,02/02/19,7,Saturday,16,NaN,rest,NaN,0.0,NaN,NaN,...,107,2099,2019-02-02,10:40,24.605447,2019-02-02,740.0,123.338142,1,2019-02-02
7,03/02/19,1,Sunday,16,NaN,long run,NaN,7.0,NaN,NaN,...,71,1350,2019-02-03,10:54,26.005446,2019-02-03,494.0,149.343588,1,2019-02-03
7,2019-02-08,1,Sunday,16,NaN,long run,NaN,7.0,NaN,NaN,...,71,1350,2019-02-03,10:54,26.005446,2019-02-03,494.0,149.343588,1,2019-02-03
7,2019-02-08,1,Sunday,16,NaN,long run,NaN,7.0,NaN,NaN,...,71,1350,2019-02-03,10:54,26.005446,2019-02-03,494.0,149.343588,1,2019-02-03


In [15]:
# Plot the plan
plan_trace = go.Scatter(x = plan['date_stamp'], y = plan['total_dist'], 
                             text=plan['text'], hoverinfo='text',
                             mode = 'lines', name='Running Plan', line = dict(color = 'grey'))

# Plot log if better than plan
log_good = go.Scatter(x = combine.loc[combine['on_track'] == 1, 'date_stamp'], 
                      y = combine.loc[combine['on_track'] == 1, 'total_run'], 
                             mode = 'lines', name='Runs logged on target', line = dict(color = 'green'))

# Plot log if worse than plan
log_bad = go.Scatter(x = combine.loc[combine['on_track'] == 0, 'date_stamp'], 
                     y = combine.loc[combine['on_track'] == 0, 'total_run'], 
                             mode = 'lines', name='Runs logged under target', line = dict(color = 'red'))

layout = go.Layout(title='Distance Covered by Running Plan', yaxis=dict(title='Distance Covered (km)'), 
                 shapes= [{'type': 'line','x0': pd.Timestamp(datetime.datetime.today().date()),'y0': 0,
                           'x1': pd.Timestamp(datetime.datetime.today().date()),'y1': max(plan['total_dist']),
                           'line': {'color': 'grey', 'dash': 'dot'}}], 
                annotations=[dict(x=datetime.date(2019, 5, 19), y=400, text='* Race Day *', font=dict(color = "gold"), showarrow=False)])

fig = go.Figure([log_good, log_bad, plan_trace], layout)
py.iplot(fig, filename='Running-Goal')

Save to the figures folder. 

In [ ]:
pio.write_image(fig, './figs/running_plan.png')

-----

# Improvement in Pace

Covering distance is one thing, but I am also interested in seeing if my running actually improves over time. To do this I will look into my pace. Another option would be to analyse HR, but I have not synced that yet so its on the to do. 

In [ ]:
runs['pace (m/s)'] = (runs['distance'] *1000)/runs['secs'] 
runs.head()

In [ ]:
tr = go.Scatter(x = runs['date_stamp'], y = runs['pace (m/s)'], 
                             text=runs['pace (m/s)'], hoverinfo='text',
                             mode = 'lines', name='Pace over time', line = dict(color = 'magenta'))

layout = go.Layout(title='Pace over time', yaxis=dict(title='Pace (m/s)'))

fig = go.Figure([tr], layout)

py.iplot(fig, filename='Running-Pace')

pio.write_image(fig, './figs/running_pace.png')

But this obviously doesn't take into account distance covered. I would hope I can run the 100m at a faster pace than 10km. So lets look at pace for distnace covered. 

In [ ]:
data = [go.Bar(x=runs['distance'], y=runs['pace (m/s)'])]
layout = go.Layout(title='Pace for each distance covered', yaxis=dict(title='Pace (m/s)'), 
                   xaxis=dict(title='Distance (km)'))

fig = go.Figure(data, layout)

In [ ]:
py.iplot(fig, filename='running-bar')

pio.write_image(fig, './figs/running_bar.png')